In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets,transforms

from torch.utils.data import DataLoader,random_split

In [ ]:
transform = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

In [ ]:
dir='data_NN'
dataset=datasets.ImageFolder(dir,transform=transform)

In [ ]:
tr=int(0.9*len(dataset))
te=len(dataset)-tr_q

In [ ]:
train, test = random_split(dataset, [tr, te])
train_loader = DataLoader(train,batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

In [ ]:
class BartHomer(nn.Module):
    def __init__(self):
        super(BartHomer,self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*64*3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 16)
        self.fc4 = nn.Linear(16,2)
    def forward(self,x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.softmax(self.fc4(x), dim=1)
        return x

In [ ]:
model=BartHomer()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs=12
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

In [ ]:
model.eval()
cor = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        cor += (predicted == labels).sum().item()

accuracy = 100 * cor / total
print(f'Accuracy on test set: {accuracy:.2f}%')